In [10]:
import os
import zipfile
import requests
import hashlib
import json

# Часть 1: Анализ киберугроз с использованием VirusTotal API

def unpack_archive(archive_path, password):
    """Распаковывает архив."""
    try:
        with zipfile.ZipFile(archive_path, 'r') as zip_ref:
            zip_ref.extractall(path='extracted_files', pwd=password.encode())
        print("Архив успешно распакован.")
    except FileNotFoundError:
        print(f"Ошибка: Файл {archive_path} не найден.")
    except RuntimeError as e:
        print(f"Ошибка при распаковке: {e}")

def get_file_hash(file_path):
    """Получает SHA256 хэш файла."""
    file_path = file_path.replace('\\', '/')
    if os.path.exists(file_path) and os.access(file_path, os.R_OK):
        try:
            with open(file_path, 'rb') as f:
                return hashlib.sha256(f.read()).hexdigest()
        except Exception as e:
            print(f"Ошибка при чтении файла {file_path}: {e}")
            return None
    else:
        print(f"Файл {file_path} недоступен или не существует.")
        return None

def scan_file(file_path, api_key):
    """Сканирует файл через VirusTotal API."""
    file_hash = get_file_hash(file_path)
    if file_hash is None:
        return None
    url = 'https://www.virustotal.com/vtapi/v2/file/report'
    params = {'apikey': api_key, 'resource': file_hash}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Ошибка запроса к VirusTotal API: {response.status_code}")
        return None

def process_results(result):
    """Обрабатывает результаты сканирования."""
    try:
        detected_by = [av for av, res in result['scans'].items() if res['detected']]
        return detected_by
    except KeyError:
        print("Некорректный формат ответа от API.")
        return []

# Основной код для части 1
archive_path = r'C:/dev/HSE/Homework/Homework/protected_archive.zip'
password = 'netology'
vt_api_key = '5b108d4a6a0189e3c0eb0411c2232b1760e344abd76c9759b765b07b2346900c'
av_list = ['Fortinet', 'McAfee', 'Yandex', 'Sophos']

unpack_archive(archive_path, password)

if not os.path.exists('extracted_files'):
    print("Директория extracted_files не создана.")
else:
    extracted_files = os.listdir('extracted_files')
    if not extracted_files:
        print("Директория extracted_files пуста.")
    else:
        print(f"Найдены файлы: {', '.join(extracted_files)}")

for file in extracted_files:
    file_path = os.path.join('extracted_files', file)
    
    if not os.path.exists(file_path) or not os.access(file_path, os.R_OK):
        print(f"Файл {file} недоступен для анализа.")
        continue
    
    result = scan_file(file_path, vt_api_key)
    
    if result:
        print(f"\nФайл: {file}")
        detected_by = process_results(result)
        
        # Вывод списка антивирусов, которые обнаружили угрозу
        if detected_by:
            print(f"Обнаружено антивирусами: {', '.join(detected_by)}")
        
        # Сравнение с заданным списком антивирусов
        print("Сравнение с заданным списком:")
        for av in av_list:
            print(f"{av}: {'Обнаружено' if av in detected_by else 'Не обнаружено'}")

# Часть 2: Анализ уязвимостей с использованием базы данных Vulners

def search_vulnerabilities(program, version, api_key):
    """Проверяет наличие уязвимостей через Vulners API."""
    query = f'"{program} {version}"'
    url = 'https://vulners.com/api/v3/search/lucene/'
    params = {
        'apiKey': api_key,
        'query': query,
        'size': 100  # Количество результатов
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Ошибка запроса к Vulners API: {response.status_code}")
        return None

vulners_api_key = 'RJ2KJIHIWSXHVDU0UJ7OSZS5R45C4BP22XQZ9VRWPCNSASQ7OX7L6CQQA6KY6KEN'
software_list = [
    {"Program": "LibreOffice", "Version": "6.0.7"},
    {"Program": "7zip", "Version": "18.05"},
    {"Program": "Adobe Reader", "Version": "2018.011.20035"},
    {"Program": "nginx", "Version": "1.14.0"},
    {"Program": "Apache HTTP Server", "Version": "2.4.29"},
    {"Program": "DjVu Reader", "Version": "2.0.0.27"},
    {"Program": "Wireshark", "Version": "2.6.1"},
    {"Program": "Notepad++", "Version": "7.5.6"},
    {"Program": "Google Chrome", "Version": "68.0.3440.106"},
    {"Program": "Mozilla Firefox", "Version": "61.0.1"}
]

for software in software_list:
    program = software['Program']
    version = software['Version']
    
    result = search_vulnerabilities(program, version, vulners_api_key)
    
    if result and result.get('data'):
        vulnerabilities = result['data']['search']
        
        cve_list = []
        exploit_count = 0
        
        for vuln in vulnerabilities:
            cve_list.extend(vuln['_source'].get('cvelist', []))
            if vuln['_source'].get('exploit_available', False):
                exploit_count += 1
        
        print(f"\nАнализ ПО: {program} {version}")
        
        # Вывод списка CVE
        if cve_list:
            print(f"Список CVE: {', '.join(cve_list)}")
        
        # Информация об эксплойтах
        print(f"Найдено эксплойтов: {exploit_count}" if exploit_count > 0 else "Информация об эксплойтах: Нет данных")


Архив успешно распакован.
Найдены файлы: invoice-42369643.html, protected_archive.zip

Файл: invoice-42369643.html
Обнаружено антивирусами: ALYac, AVG, Arcabit, Avast, BitDefender, CTX, DrWeb, ESET-NOD32, Emsisoft, FireEye, Fortinet, GData, Google, Ikarus, Kaspersky, Kingsoft, Lionic, MicroWorld-eScan, Microsoft, Symantec, VIPRE, Varist, ViRobot, alibabacloud
Сравнение с заданным списком:
Fortinet: Обнаружено
McAfee: Не обнаружено
Yandex: Не обнаружено
Sophos: Не обнаружено

Файл: protected_archive.zip
Сравнение с заданным списком:
Fortinet: Не обнаружено
McAfee: Не обнаружено
Yandex: Не обнаружено
Sophos: Не обнаружено

Анализ ПО: LibreOffice 6.0.7
Список CVE: CVE-2018-16858, CVE-2018-16858, CVE-2020-12801, CVE-2020-12803, CVE-2022-26305, CVE-2022-26306, CVE-2022-26307, CVE-2022-3140, CVE-2022-38745, CVE-2021-25636, CVE-2019-9854, CVE-2019-9848, CVE-2019-9849, CVE-2019-9850, CVE-2019-9851, CVE-2019-9852, CVE-2017-12607, CVE-2017-12608, CVE-2017-7870, CVE-2017-8358, CVE-2018-10119, CVE